In [84]:
from bs4 import BeautifulSoup
import requests
import re
import time
import datetime
import csv
import smtplib
import pandas as pd

In [17]:
URL = 'https://www.amazon.com/dp/B0BSHF7WHW'

# headers is the info of the user system.
# One can find user agent it in google developer tools under console by typing in 
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36'

# full headers info you can find in https://httpbin.org/get
# one needs to get accept, accpet-encoding, accept-language, upgrade-insecure-requests, user-agent
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
    "Accept-Encoding": "gzip, deflate, br, zstd", 
    "Accept-Language": "en-US,en;q=0.9", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
  }

In [23]:
target_url="https://api.scrapingdog.com/scrape?api_key=xxxxxxxxxxxxxxxxxxxx&url=https://www.amazon.com/dp/B0BSHF7WHW&dynamic=false"


In [24]:
page = requests.get(target_url)

In [28]:
page = requests.get(URL, headers=headers)

In [29]:
page

<Response [200]>

In [30]:
soup1 = BeautifulSoup(page.content, 'html.parser')

soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

In [34]:
title = soup2.find('h1',{'id':'title'}).text.strip()
title

'Apple 2023 MacBook Pro laptop M2 Pro chip with 12‑core CPU and 19‑core GPU: 16.2-inch Liquid Retina XDR display, 16GB Unified Memory, 1TB SSD storage. Works with iPhone/iPad; Space Gray'

In [36]:
allimages = soup2.find_all("div",{"class":"imgTagWrapper"})
print(len(allimages))

3


In [42]:
images = re.findall('"hiRes":"(.+?)"', page.text)
images

['https://m.media-amazon.com/images/I/61fd2oCrvyL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/71ZAeHYYlHL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/61C9irOOQVL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/81j1XDaqcML._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/81OBjqVEGwL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/61FhAanYdhL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/61bwiPRcv2L._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/71-uDe4TN1L._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/61C9irOOQVL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/81KJXaswhrL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/81OBjqVEGwL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/61CdlgWgngL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/61EXF2Vr9LL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/61fd2oCrvyL._AC_SL1500_.jpg',
 'https://m.media-amazon.com/images/I/71ZAeHYYlH

In [57]:
price=soup2.find("td",{"class":"a-span12"}).find("span").find("span").text.strip()

In [58]:
print(price)

$2,699.00


In [60]:
rating = soup2.find("i",{"class":"a-icon-star"}).text.strip()

In [61]:
rating

'4.7 out of 5 stars'

In [68]:
specs_obj = {}
specs = soup2.find_all("tr",{"class":"a-spacing-small"})
for u in range(0,len(specs)):
    spanTags = specs[u].find_all("span")
    specs_obj[spanTags[0].text.strip()]=spanTags[1].text.strip()

In [69]:
specs_obj

{'Brand': 'Apple',
 'Model Name': 'MacBook Pro',
 'Screen Size': '16.2 Inches',
 'Color': 'Space Gray',
 'Hard Disk Size': '1 TB',
 'CPU Model': 'Unknown',
 'Ram Memory Installed Size': '16 GB',
 'Operating System': 'Mac OS',
 'Special Feature': 'Anti Glare Coating',
 'Graphics Card Description': 'Integrated'}

In [70]:
info = {'title' : title, 'price' : price, 'images' : images, 'specs' : specs_obj}

In [71]:
info

{'title': 'Apple 2023 MacBook Pro laptop M2 Pro chip with 12‑core CPU and 19‑core GPU: 16.2-inch Liquid Retina XDR display, 16GB Unified Memory, 1TB SSD storage. Works with iPhone/iPad; Space Gray',
 'price': '$2,699.00',
 'images': ['https://m.media-amazon.com/images/I/61fd2oCrvyL._AC_SL1500_.jpg',
  'https://m.media-amazon.com/images/I/71ZAeHYYlHL._AC_SL1500_.jpg',
  'https://m.media-amazon.com/images/I/61C9irOOQVL._AC_SL1500_.jpg',
  'https://m.media-amazon.com/images/I/81j1XDaqcML._AC_SL1500_.jpg',
  'https://m.media-amazon.com/images/I/81OBjqVEGwL._AC_SL1500_.jpg',
  'https://m.media-amazon.com/images/I/61FhAanYdhL._AC_SL1500_.jpg',
  'https://m.media-amazon.com/images/I/61bwiPRcv2L._AC_SL1500_.jpg',
  'https://m.media-amazon.com/images/I/71-uDe4TN1L._AC_SL1500_.jpg',
  'https://m.media-amazon.com/images/I/61C9irOOQVL._AC_SL1500_.jpg',
  'https://m.media-amazon.com/images/I/81KJXaswhrL._AC_SL1500_.jpg',
  'https://m.media-amazon.com/images/I/81OBjqVEGwL._AC_SL1500_.jpg',
  'https:

In [72]:
today = datetime.date.today()

In [73]:
today

datetime.date(2024, 8, 10)

In [80]:
header = ['Title', 'Price', 'Date']
data = [title, price, today]

with open('AmazonWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    # writer.writerow(header)
    writer.writerow(data)

In [81]:
df = pd.read_csv(r'.\AmazonWebScraperDataset.csv')

print(df)

                                               Title      Price        Date
0  Apple 2023 MacBook Pro laptop M2 Pro chip with...  $2,699.00  2024-08-10


In [83]:
# can receive email if say price hits certain level, following function can be set to be trigerred

def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    #server.starttls()
    server.ehlo()
    server.login('AlexTheAnalyst95@gmail.com','xxxxxxxxxxxxxx')
    
    subject = "The laptop you want is below $2400! Now is your chance to buy!"
    body = "Sharan, This is the moment we have been waiting for. Now is your chance to pick it. Link here: https://www.amazon.com/dp/B0BSHF7WHW"
   
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'sharanbennur9900@gmail.com',
        msg
    )

In [92]:
def check_price():
    
    URL = 'https://www.amazon.com/dp/B0BSHF7WHW'
    headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7", 
    "Accept-Encoding": "gzip, deflate, br, zstd", 
    "Accept-Language": "en-US,en;q=0.9", 
    "Upgrade-Insecure-Requests": "1", 
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
    }

    page = requests.get(URL, headers=headers)

    soup1 = BeautifulSoup(page.content, 'html.parser')

    soup2 = BeautifulSoup(soup1.prettify(), 'html.parser')

    title = soup2.find('h1',{'id':'title'}).text.strip()

    rating = soup2.find("i",{"class":"a-icon-star"}).text.strip()

    header = ['Title', 'Price', 'Date']
    today = datetime.date.today()
    data = [title, price, today]

    with open('AmazonWebScraperDataset.csv', 'a+', newline='', encoding='UTF8') as f:
        writer = csv.writer(f)
        # writer.writerow(header)
        writer.writerow(data)

    if price[1:0]>'2400':
        send_mail()

    return 0
    
    

    

In [ ]:
# Runs check_price after a set time and inputs data into your CSV
# combine the above scraping steps into check_price() function to automatically collect price changes and write into the csv file
while(True):
    check_price()
    time.sleep(86400)